In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.feature as feat
import skimage.data as data
from skimage import io, util
import skimage.color as color
import matplotlib.pyplot as plt

In [ ]:
# img = io.imread('images/test/giant-rubber-duck.avif')
img = io.imread('images/board.jpg')
plt.imshow(util.img_as_ubyte(img)); 
plt.axis('off'); 
plt.show()

In [ ]:
gray_image = color.rgb2gray(img)
# low_threshold and high_threshold defaults are .1 and .2
edges = feat.canny(gray_image, low_threshold=.1, high_threshold=0.2, sigma=2)
plt.axis('off')
plt.imshow(edges, cmap='gray')

In [ ]:
hsv = color.rgb2hsv(img)
H = hsv[..., 0]
S = hsv[..., 1]
V = hsv[..., 2]

mask = ((H >= 0.04) & (H <= 0.17)) & (S >= 0.4) & (V >= 0.3)

gray = color.rgb2gray(img)
gray_masked = gray * mask
edges = feat.canny(gray_masked, low_threshold=0.05, high_threshold=0.15, sigma=2)

overlay = util.img_as_ubyte(img.copy())
overlay[edges] = [255, 0, 255]

from matplotlib.colors import ListedColormap
purple_cmap = ListedColormap(['#FF0000', '#FFFF00'])

fig, axes = plt.subplots(1, 3, figsize=(15,5))
axes[0].imshow(util.img_as_ubyte(img)); axes[0].axis('off')
axes[0].set_title('RGB image')
axes[1].imshow(mask, cmap=purple_cmap, interpolation='nearest')
axes[1].axis('off')
axes[1].set_title('Yellow/Orange mask')
axes[2].imshow(overlay); axes[2].axis('off'); axes[2].set_title('Edges on yellow/orange (magenta overlay)')
plt.show()

## Detecting krokinole pieces (circular pieces)
This notebook cell explains and runs a Hough Circle based detector to find circular pieces on the krokinole board photo.

How it works:
- Convert the image to grayscale and detect edges (Canny).
- Run the Hough Circle transform over a range of radii estimated from image size.
- Keep the strongest detected circles and overlay them on the original image.

Run the next cell to execute the detector. If you get too many/too few detections, open the "Tuning notes" cell for parameters to change.

In [ ]:
# Hough circle detection for krokinole pieces using scikit-image
from skimage import feature, color, util, transform, draw
import numpy as np
import matplotlib.pyplot as plt

# Ensure image is available as `img` from earlier cell; if not, load it here (uncomment):
# img = io.imread('images/test/giant-rubber-duck.avif')
# img = io.imread('images/board.jpg')

gray = color.rgb2gray(img)
# edge detection (tune sigma as needed)
edges = feature.canny(gray, sigma=2)

# Estimate radii range from image size:
h, w = gray.shape
diag = int(np.hypot(h, w))
# pieces are relatively small compared to board; choose radii as pixels:
min_r = max(8, int(min(h,w) * 0.02))
max_r = max(min_r+1, int(min(h,w) * 0.06))
radii = np.arange(min_r, max_r, 2)  # step 2 px
print(f"image size: {w}x{h}, searching radii {min_r}..{max_r}")

# Run Hough Circle transform
hough_res = transform.hough_circle(edges, radii)

# Select the most prominent circles
accums, cx, cy, radii_peaks = transform.hough_circle_peaks(hough_res, radii, total_num_peaks=40)

# Filter duplicates / nearby centers: keep stronger ones with non-max suppression
selected = []
for acc, x0, y0, r0 in zip(accums, cx, cy, radii_peaks):
    keep = True
    for (_, xx, yy, rr) in selected:
        if np.hypot(xx - x0, yy - y0) < rr*0.6:
            keep = False
            break
    if keep:
        selected.append((acc, x0, y0, r0))

# Draw results on overlay
overlay = util.img_as_ubyte(img.copy())
fig, ax = plt.subplots(1, figsize=(8,8))
ax.imshow(overlay)
for acc, x0, y0, r0 in selected:
    circy, circx = draw.circle_perimeter(int(y0), int(x0), int(r0), shape=overlay.shape)
    overlay[circy, circx] = (255, 0, 0)  # red perimeter
    ax.plot(x0, y0, 'go', markersize=4)
ax.set_title(f"Detected circles: {len(selected)}")
ax.axis('off')
plt.imshow(overlay)
plt.show()

# Print detected circle parameters
for i, (_, x0, y0, r0) in enumerate(selected):
    print(f"circle {i}: center=({int(x0)},{int(y0)}), r={int(r0)})")